# MNIST（手写数字识别）
问题：
1. 输入：28x28的灰阶图像（flatten为784的向量）
2. 输出：对应的数字（one-hot vector每个下标对应的数字的概率值）
3. solution: 使用线性模型获得输入图像的evidence值，使用softmax regression来训练

In [41]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Step 1. 
Define the model. (i.e. How to depict the structure of dataset)

In [42]:
x = tf.placeholder(tf.float32, [None, 28*28]) # None means any number in this dimension
W = tf.Variable(tf.zeros([28*28, 10])) # the parameter
b = tf.Variable(tf.zeros([10])) # the bias

# tf.nn是神经网络的支持库，包含神经网络操作
y = tf.nn.softmax(tf.matmul(x, W)+b) # the regrassion model

### Step 2. 
Define the measurement of the model. (i.e. How bad it is)

Here, use [cross entropy](http://colah.github.io/posts/2015-09-Visual-Information/): \\(-\sum{y^{'}log(y)}\\)

In [43]:
y_ = tf.placeholder(tf.float32, [None, 10])
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduce_indices=[1]))

# use library utility to have a more stable computation
# must specify the raw log probability and label
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = tf.matmul(x, W)+b, labels = y_))

### step3.
Define the training method.

[由于tensorflow已经知道整个计算图，于是可以自己针对优化的loss目标来调整参数]

In [44]:
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

### Step4.
Start training. Stochastic gradient descent: effective and efficient.

#### Global Initializer
将当前发现的需要初始化的variable放入group中。<b>其后面创建的variable不会背放入该组</b>

In [45]:
session = tf.InteractiveSession() # 产生一个交互式的训练会话
tf.global_variables_initializer().run() #初始化所有的全局变量

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(1000)
    session.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})

### Step5.
Evaluate the trained model.

In [46]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("accuracy: {0:.2f}%".format(session.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels})*100))

accuracy: 90.22%
